# THE PLAN
1. A function, that will accept minio url
2. Download the video from minio to a temp directory
3. Extract Audio
4. Load OpenAI's Whisper model
5. Covert Audio to Text
6. append .srt artifacts,i.e, every sentence is a new line and has timestamps (maybe?)
7. Upload the .srt file to minio
8. Use the text to generate summary, and chapters for the video
9. Summary can be stored in DB as video.description (so 2 things,1> pass the video id & 2> move DB to PG or smthing)
10. IDK how to store the chapters, maybe in DB

In [5]:
%pip install minio -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
MINIO_IP="192.168.0.118"
MINIO_PORT="9010"
MINIO_ACCESS_KEY="oloom_key"
MINIO_SECRET_KEY="KhGjEmfDW07wMN34SYKJkv539o7Rfq8yoGK0efeS"

In [27]:
minio_client = Minio( f"{MINIO_IP}:{MINIO_PORT}", access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False )
minio_client.bucket_exists("oloom")


True

In [ ]:
from tempfile import mkdtemp
import shutil
from minio import Minio
import os

class VideoProcessor:
    def __init__(self,video_id,file_name):
        temp_dir = mkdtemp(prefix=f"oloom_")
        print(f"Temp dir: {temp_dir}")
        print(f"Temp video path: {temp_dir}/{file_name}")
        self.video_id = video_id
        self.file_name = file_name
        self.temp_dir = temp_dir
        self.video_path = f"{temp_dir}/{file_name}"
        self.minio_client = Minio(
            f"{MINIO_IP}:{MINIO_PORT}",
            access_key=MINIO_ACCESS_KEY,
            secret_key=MINIO_SECRET_KEY,
            secure=False
        )
        if not self.minio_client.bucket_exists("oloom"):
            raise Error("Minio bucket does not exist")
        
        self.download_video()
        # self.cleanup()

    def download_video(self):
        try:
            self.minio_client.fget_object(bucket_name="oloom",object_name=self.file_name,file_path=self.video_path)
            return True
        except Exception as e:
            logger.error(f"Error downloading video: {e}")
            raise e

    def cleanup(self):
        shutil.rmtree(self.temp_dir)

In [53]:
VideoProcessor = VideoProcessor("123","920bdf9e-159d-447b-9720-c7575436cbe8-1 Hour of Berserk Music (Theme of Guts).mp4")

Temp dir: /var/folders/b9/x0sm9z492qd6qs1dyjpcyydw0000gn/T/oloom_1dwmm5a1
Temp video path: /var/folders/b9/x0sm9z492qd6qs1dyjpcyydw0000gn/T/oloom_1dwmm5a1/920bdf9e-159d-447b-9720-c7575436cbe8-1 Hour of Berserk Music (Theme of Guts).mp4
